# Simple Classifiers and preprocessing of the data

In [9]:
import sys
sys.path.append('..')

import os
import matplotlib.pyplot as plt
import pandas as pd
from src.plot_utils import hist, count, scatter
from dotenv import load_dotenv
from sklearn.model_selection import train_test_split 

SEED = 42


## Importing of the data and spliting

next, we will import the dataset:

In [4]:
CSV_PATH = '../data/chartex_clean.csv'

df = pd.read_csv(CSV_PATH)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4447 entries, 0 to 4446
Data columns (total 25 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   track_name                    4446 non-null   object 
 1   track_pop                     4447 non-null   int64  
 2   artist                        4446 non-null   object 
 3   artist_pop                    4447 non-null   int64  
 4   album                         4446 non-null   object 
 5   length                        4447 non-null   int64  
 6   danceability                  4443 non-null   float64
 7   energy                        4443 non-null   float64
 8   key                           4443 non-null   float64
 9   loudness                      4443 non-null   float64
 10  mode                          4443 non-null   float64
 11  speechiness                   4443 non-null   float64
 12  acousticness                  4443 non-null   float64
 13  ins

next, we will split the target feature (track_pop) from the rest of the features and split to train and test sets:

In [6]:
X, y = df.drop(['track_pop', 'length'], axis = 1).values , df['track_pop'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = SEED, test_size=0.25)